# Capturing Blood Tissue in GTEx Data

**Gregory Way, 2018**

When viewing sample correlation differences across z dimensions stratified by tissue-type, we often observed a rapid increase in correlation after increasing model capacity by one.

For example, the ability of variational autoencoders to capture blood tissue correlation jumps by nearly 50% between bottleneck dimensions 2 and 3 (see below).

![sample-correlation_Blood_GTEX_signal_pearson.png](https://raw.githubusercontent.com/greenelab/BioBombe/master/4.analyze-components/figures/GTEX/sample-correlation/sample-type/sample-correlation_Blood_GTEX_signal_pearson.png)

## Procedure

In the following notebook, we extract two representative weight matrices for VAE latent space dimensions 2 and 3.
We apply two compression feature interpretaion approaches to the weight vectors for each latent space feature. 

1. Our matrix interpretation approach
2. Overrepresentation Tests using high weight genes

In both approaches we use genesets derived in the xCell paper that represent cell-types ([Aran et al. 2017](https://doi.org/10.1186/s13059-017-1349-1))

We output the results of both approaches and analyze the results in subsequent notebooks.

In [1]:
import os
import sys
import glob
import pandas as pd
import numpy as np

sys.path.append('../scripts')
from latent import latentModel, load_hetnets, parse_gmt, run_overrepresentation

In [2]:
np.random.seed(123)

In [3]:
metaedge = 'GpXCELL'

In [4]:
base_dir = os.path.join('..', '2.ensemble-z-analysis', 'results',
                        'GTEX_results', 'ensemble_z_matrices')

In [5]:
sample_weight_2 = os.path.join(base_dir, 'gtex_components_2', 'model_908341_weight_matrix.tsv.gz')
sample_weight_3 = os.path.join(base_dir, 'gtex_components_3', 'model_908341_weight_matrix.tsv.gz')

In [6]:
sample_weight_2_df = pd.read_table(sample_weight_2, index_col=0)
sample_weight_3_df = pd.read_table(sample_weight_3, index_col=0)

sample_weight_2_df = sample_weight_2_df.loc[:, sample_weight_2_df.columns.str.contains('vae')]
sample_weight_3_df = sample_weight_3_df.loc[:, sample_weight_3_df.columns.str.contains('vae')]

# Recode column names
sample_weight_2_df.columns = sample_weight_2_df.columns + '_two'
sample_weight_3_df.columns = sample_weight_3_df.columns + '_three'

sample_weight_2_df.head()

,vae_0_two,vae_1_two
gene_id,,
3043,0.176201,-0.165548
3040,0.163633,-0.163657
6280,-0.007069,-0.225171
5644,0.219859,-0.106132
2512,-0.059835,0.051747


In [7]:
combined_model = sample_weight_2_df.merge(sample_weight_3_df,
                                          left_index=True,
                                          right_index=True)
combined_model.index = combined_model.index.astype('str')
combined_model.head(2)

,vae_0_two,vae_1_two,vae_0_three,vae_1_three,vae_2_three
gene_id,,,,,
3043,0.176201,-0.165548,0.314363,-0.170519,0.021420
3040,0.163633,-0.163657,0.294492,-0.188674,0.008922


In [8]:
# Load hetnets for the given metaedge for rapid latent feature interpretation
hetnets = load_hetnets(
    hetnet_file='../3.build-hetnets/hetnets/interpret_hetnet.json.bz2',
    permuted_directory='../3.build-hetnets/hetnets/permuted/',
    subset_genes=combined_model.index,
    metaedge_abbrev=metaedge
   )

## Apply Interpret Compression Approach

In [9]:
%%time

mult_results = {}
all_list = []
for model in hetnets.keys():
    hetnet = hetnets[model]
    mult_results[model] = combined_model.T @ hetnet
    long_result = mult_results[model].reset_index().melt(id_vars=['index'])
    long_result = long_result.assign(model=model)
    if model != 'real':
        long_result = long_result.assign(model_type='permuted')
    else:
        long_result = long_result.assign(model_type='real')
    all_list.append(long_result)

CPU times: user 6.3 s, sys: 18.2 ms, total: 6.32 s
Wall time: 8.86 s


In [10]:
all_df = pd.concat(all_list)
all_df.value = all_df.value.astype(float)
all_df = all_df.rename(columns={'index': 'full_feature', 'value': 'raw_score'})
all_df.head()

,full_feature,variable,raw_score,model,model_type
0,vae_0_two,Adipocytes_ENCODE_1,-0.580380,0,permuted
1,vae_1_two,Adipocytes_ENCODE_1,-0.092745,0,permuted
2,vae_0_three,Adipocytes_ENCODE_1,-0.390124,0,permuted
3,vae_1_three,Adipocytes_ENCODE_1,-0.430026,0,permuted
4,vae_2_three,Adipocytes_ENCODE_1,-0.081764,0,permuted


In [11]:
all_group = all_df.groupby(['model_type', 'full_feature', 'variable'])
all_group_mean = all_group.mean().reset_index()

real_df = all_group_mean.query("model_type == 'real'")
permuted_mean = all_group_mean.query("model_type == 'permuted'")

permuted_std = (
    all_group
    .std()
    .reset_index()
    .query("model_type == 'permuted'")
)

In [12]:
z_score = (real_df.reset_index(drop=True).raw_score - permuted_mean.raw_score) / permuted_std.raw_score
real_df = real_df.reset_index(drop=True).assign(z_score=z_score)

real_df.head()

,model_type,full_feature,variable,raw_score,z_score
0,real,vae_0_three,Adipocytes_ENCODE_1,0.251069,2.423840
1,real,vae_0_three,Adipocytes_ENCODE_2,0.265078,2.788173
2,real,vae_0_three,Adipocytes_ENCODE_3,0.314689,1.930844
3,real,vae_0_three,Adipocytes_FANTOM_1,0.437572,3.728542
4,real,vae_0_three,Adipocytes_FANTOM_2,0.437572,2.318366


In [13]:
# Output z scores per xCell genesets for all VAE features (5 total)
file = os.path.join('results', 'gtex_vae_example_interpret_compression.tsv')
real_df.sort_values(by='z_score').to_csv(file, sep='\t', index=False)

In [14]:
# Determine the most distinguishing features between the two models
feature_info_df = (
    pd.DataFrame(real_df['full_feature'].str.split('_').values.tolist(),
                 columns=['algorithm', 'feature', 'model_z'])

)

In [15]:
feature_info_df = pd.concat([real_df, feature_info_df], axis=1)
feature_info_df = feature_info_df.assign(abs_z_score = feature_info_df.z_score.abs())

In [16]:
feature_info_df = (
    feature_info_df.groupby(['variable', 'model_z'])
    .mean()
    .reset_index()
    .pivot(index='variable', columns='model_z', values = 'abs_z_score')
)

feature_info_df = (
    feature_info_df.assign(abs_diff = (feature_info_df.three - feature_info_df.two).abs())
    .sort_values(by='abs_diff', ascending=False)
)

feature_info_df.head()

model_z,three,two,abs_diff
variable,,,
Skeletal-muscle_FANTOM_2,7.946984,15.068043,7.121059
Monocytes_FANTOM_2,6.910606,0.709876,6.200731
Monocytes_IRIS_3,7.092820,1.342540,5.750279
Astrocytes_FANTOM_3,4.243827,9.696995,5.453168
Monocytes_NOVERSHTERN_1,7.413432,1.991430,5.422002


In [17]:
# Output the most distinguishing features considering all features within each z dimension
file = os.path.join('results', 'gtex_vae_example_differentiating_features.tsv')
feature_info_df.to_csv(file, sep='\t')

## Perform overrepresentation tests on the same features

### Split into positive and negative tails and extract high weight genes

In [18]:
geneset_file = os.path.join('..', '3.build-hetnets', 'data', 'xcell_all_entrez.gmt')
xcell_genesets_gmt = parse_gmt(gene_sets=[geneset_file])
len(xcell_genesets_gmt)

489

In [19]:
vae_2_lm = latentModel(filename=sample_weight_2,
                       z_dim=2,
                       dataset_name='GTEX',
                       algorithm_name='VAE',
                       weight_seed='908341',
                       shuffled_true=False)

vae_3_lm = latentModel(filename=sample_weight_3,
                       z_dim=3,
                       dataset_name='GTEX',
                       algorithm_name='VAE',
                       weight_seed='908341',
                       shuffled_true=False)

In [20]:
vae_2_lm.get_high_weight_genes()
vae_3_lm.get_high_weight_genes()

In [21]:
background_genes = []
for xcell_name, xcell_gene_set in xcell_genesets_gmt.items():
    background_genes += xcell_gene_set

In [22]:
%%time

gene_list = vae_2_lm.w_df[vae_2_lm.pos_high_w_df.vae_0].index.tolist()
vae_feature_0_zdim_2_pos = run_overrepresentation(gene_list=gene_list,
                                                  gene_set_dict=xcell_genesets_gmt,
                                                  background_genes=background_genes)

gene_list = vae_2_lm.w_df[vae_2_lm.neg_high_w_df.vae_0].index.tolist()
vae_feature_0_zdim_2_neg = run_overrepresentation(gene_list=gene_list,
                                                  gene_set_dict=xcell_genesets_gmt,
                                                  background_genes=background_genes)

gene_list = vae_2_lm.w_df[vae_2_lm.pos_high_w_df.vae_1].index.tolist()
vae_feature_1_zdim_2_pos = run_overrepresentation(gene_list=gene_list,
                                                  gene_set_dict=xcell_genesets_gmt,
                                                  background_genes=background_genes)

gene_list = vae_2_lm.w_df[vae_2_lm.neg_high_w_df.vae_1].index.tolist()
vae_feature_1_zdim_2_neg = run_overrepresentation(gene_list=gene_list,
                                                  gene_set_dict=xcell_genesets_gmt,
                                                  background_genes=background_genes)

gene_list = vae_3_lm.w_df[vae_3_lm.pos_high_w_df.vae_0].index.tolist()
vae_feature_0_zdim_3_pos = run_overrepresentation(gene_list=gene_list,
                                                  gene_set_dict=xcell_genesets_gmt,
                                                  background_genes=background_genes)

gene_list = vae_3_lm.w_df[vae_3_lm.neg_high_w_df.vae_0].index.tolist()
vae_feature_0_zdim_3_neg = run_overrepresentation(gene_list=gene_list,
                                                  gene_set_dict=xcell_genesets_gmt,
                                                  background_genes=background_genes)

gene_list = vae_3_lm.w_df[vae_3_lm.pos_high_w_df.vae_1].index.tolist()
vae_feature_1_zdim_3_pos = run_overrepresentation(gene_list=gene_list,
                                                  gene_set_dict=xcell_genesets_gmt,
                                                  background_genes=background_genes)

gene_list = vae_3_lm.w_df[vae_3_lm.neg_high_w_df.vae_1].index.tolist()
vae_feature_1_zdim_3_neg = run_overrepresentation(gene_list=gene_list,
                                                  gene_set_dict=xcell_genesets_gmt,
                                                  background_genes=background_genes)

gene_list = vae_3_lm.w_df[vae_3_lm.pos_high_w_df.vae_2].index.tolist()
vae_feature_2_zdim_3_pos = run_overrepresentation(gene_list=gene_list,
                                                  gene_set_dict=xcell_genesets_gmt,
                                                  background_genes=background_genes)

gene_list = vae_3_lm.w_df[vae_3_lm.neg_high_w_df.vae_2].index.tolist()
vae_feature_2_zdim_3_neg = run_overrepresentation(gene_list=gene_list,
                                                  gene_set_dict=xcell_genesets_gmt,
                                                  background_genes=background_genes)

CPU times: user 4.56 s, sys: 18.4 ms, total: 4.57 s
Wall time: 6.79 s


In [23]:
vae_feature_0_zdim_2_pos = vae_feature_0_zdim_2_pos.assign(feature='vae_0_two', tailed='pos')
vae_feature_0_zdim_2_neg = vae_feature_0_zdim_2_neg.assign(feature='vae_0_two', tailed='neg')
vae_feature_1_zdim_2_pos = vae_feature_1_zdim_2_pos.assign(feature='vae_1_two', tailed='pos')
vae_feature_1_zdim_2_neg = vae_feature_1_zdim_2_neg.assign(feature='vae_1_two', tailed='neg')

vae_feature_0_zdim_3_pos = vae_feature_0_zdim_3_pos.assign(feature='vae_0_three', tailed='pos')
vae_feature_0_zdim_3_neg = vae_feature_0_zdim_3_neg.assign(feature='vae_0_three', tailed='neg')
vae_feature_1_zdim_3_pos = vae_feature_1_zdim_3_pos.assign(feature='vae_1_three', tailed='pos')
vae_feature_1_zdim_3_neg = vae_feature_1_zdim_3_neg.assign(feature='vae_1_three', tailed='neg')
vae_feature_2_zdim_3_pos = vae_feature_2_zdim_3_pos.assign(feature='vae_2_three', tailed='pos')
vae_feature_2_zdim_3_neg = vae_feature_2_zdim_3_neg.assign(feature='vae_2_three', tailed='neg')

In [24]:
overrepresented_results_df = pd.concat([
    vae_feature_0_zdim_2_pos,
    vae_feature_0_zdim_2_neg,
    vae_feature_1_zdim_2_pos,
    vae_feature_1_zdim_2_neg,
    vae_feature_0_zdim_3_pos,
    vae_feature_0_zdim_3_neg,
    vae_feature_1_zdim_3_pos,
    vae_feature_1_zdim_3_neg,
    vae_feature_2_zdim_3_pos,
    vae_feature_2_zdim_3_neg
])

overrepresented_results_df.head()

,pval,odds,feature,tailed
pro-B-cells_NOVERSHTERN_2,0.220006,4.339523,vae_0_two,pos
Endothelial-cells_ENCODE_3,0.369979,2.251034,vae_0_two,pos
CD4+-T-cells_FANTOM_3,1.000000,0.732759,vae_0_two,pos
DC_FANTOM_3,0.323462,2.681992,vae_0_two,pos
MPP_BLUEPRINT_1,0.199574,2.519897,vae_0_two,pos


In [25]:
file = os.path.join('results', 'gtex_vae_example_overrepresentation.tsv')
(
    overrepresented_results_df
    .reset_index()
    .rename(columns={'index': 'variable',
                     'feature': 'full_feature'})
    .sort_values(by='pval')
    .to_csv(file, sep='\t', index=False)
)